In [ ]:
import os, shutil, sys, random, cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import History
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from random import choice

In [ ]:
dataset_dividido_folder = 'brain_tumor_dataset_dividido/'

treino_folder = 'treino/'
teste_folder = 'teste/'
#validacao_folder = 'validacao/'

dataset_dividido_folder_treino = dataset_dividido_folder + treino_folder
dataset_dividido_folder_teste = dataset_dividido_folder + teste_folder
#dataset_dividido_folder_validacao = dataset_dividido_folder + validacao_folder

# Data Augmentation

In [ ]:
datagen_treino = ImageDataGenerator(rescale = 1./255,
                                    brightness_range=(0.7, 1.3),
                                    vertical_flip=True,
                                    horizontal_flip=True,
                                    validation_split = 0.25)

#datagen_validacao = ImageDataGenerator(rescale = 1./255)
datagen_teste = ImageDataGenerator(rescale = 1./255)

In [ ]:
seed = 123

dados_treino = datagen_treino.flow_from_directory(
    dataset_dividido_folder_treino,
    target_size = (150,150),
    color_mode = "grayscale",
    batch_size = 32,
    class_mode="binary",
    subset='training',
    seed = seed)



dados_validacao = datagen_treino.flow_from_directory(
    dataset_dividido_folder_treino,
    target_size = (150,150),
    color_mode = "grayscale",
    batch_size = 32,
    class_mode="binary",
    subset='validation',
    seed = seed)

'''dados_validacao = datagen_validacao.flow_from_directory(
    dataset_dividido_folder_validacao,
    target_size = (150,150),
    color_mode = "grayscale",
    batch_size = 30,
    class_mode="binary",
    seed = seed)'''

dados_teste = datagen_teste.flow_from_directory(
    dataset_dividido_folder_teste,
    target_size = (150,150),
    color_mode = "grayscale",
    batch_size = 32,
    class_mode = 'binary'
    )

print(dados_treino.class_indices)

# Visualizar exemplo Data Augmentation

In [ ]:
#.__getitem__(batch, max de 8)[atributos/class :0 -> atributos, 1 -> class][imagens, max de 29, batch_size 30]
img = dados_treino.__getitem__(0)[0][1]
x = image.img_to_array(img)
plt.figure(0)
plt.imshow(np.squeeze(x, axis=2), cmap = "Greys")
plt.title('Imagem Inicial')
x = x.reshape((1,) + x.shape)
i = 1
#.flow -> real time data augmentation.
for batch in datagen_treino.flow(x, batch_size=1):
    plt.figure(i)
    plt.title('Imagem Gerada ' + str(i))
    imgplot = plt.imshow(image.array_to_img(batch[0]), cmap = "Greys", interpolation = "nearest")
    i += 1
    if i % 4 == 0:
        break

plt.show()

# Criar Modelo

In [ ]:
#Foi acrescentado ReduceLROnPlateau callback ao modelo.
#Aplicando ReduceLROnPlateau  reduz a taxa de aprendizagem por um fator de 2-10 quando o processo de aprendizagem estagna.
#O callback monitoriza o processo e se não houver melhorias em 'patience' epochs seguidas, a taxa de aprendizagem é reduzida.
Reduce_learning_rate = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patiente = 2, cooldown = 2, min_lr = 0.00001, verbose = 1)
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", monitor = 'val_accuracy', verbose=1, save_best_only=True)
callbacks = [checkpointer,Reduce_learning_rate]

In [ ]:
def setup_model_dynamic(num_neurons, input_size, output_size, dropout_size, activation, activation_output):
    model = Sequential()
    model.add(Conv2D(num_neurons[0],(3,3), input_shape = (150,150,1), activation = activation))
    model.add(MaxPooling2D(pool_size = (2,2)))
    for i in range(1, len(num_neurons)):
        model.add(Conv2D(num_neurons[i],(3,3), activation = activation))
        model.add(MaxPolling2D(pool_size = (2,2)))
        if dropout_size > 0: model.add(Dropout(dropout_size))
    model.add(Flatten())
    model.add(Dropout(dropout_size))
    model.add(Dense(64, activation = activation))
    model.add(Dense(1, activation= activation_output))
    
    return model

#model.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
def setup_compile_model(model, optimizer, dados_treino, dados_validacao, lr, loss, epochs, steps_per_epochs, metric):
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate = lr)
    elif optimizer == 'RMSprop':
        opt = tf.keras.optimizers.Rmsprop(learning_rate = lr)
    elif optimizer == 'sgd_momentum':
        opt = tf.keras.optimizers.SGD(learning_rate = lr, momentum = 0.9)
    else: opt = tf.keras.optimizers.Adam(learning_rate = lr)
    model.compile(loss = loss, optimizer = opt, metrics = ["accuracy"])  #['"' + metric + '"'])
    history = model.fit(dados_treino, steps_per_epoch = steps_per_epochs, epochs = epochs, validation_data = dados_validacao, callbacks = callbacks, verbose = 1)
    return model,history

In [ ]:
def run_dnn(dados_treino, dados_validacao, dados_teste, num_iterations, verbose = True):
    best_accuracy = 0
    num_neuron = [[64], [32]]
    input_size = (150,150,1)
    output_size = 1
    dropouts = [0, 0.2, 0.4, 0.5]
    metric = ['accuracy']
    activations = ['relu', 'tanh']
    activations_output = ['sigmoid']
    losses = ['binary_crossentropy']
    optimizers = [ 'adam', 'rmsprop', 'sgd_momentum']
    epoch = [30, 15, 20]
    steps_per_epochs = [5]
    learning_rates = [0.01, 0.001, 0.0001]
    
    for i in range(num_iterations):
        num_neurons = choice(num_neuron)
        dropout = choice(dropouts)
        metrics = choice(metric)
        optimizer = choice(optimizers)
        activation = choice(activations)
        activation_output = choice(activations_output)
        loss = choice(losses)
        epochs = choice(epoch)
        steps_per_epoch = choice(steps_per_epochs)
        learning_rate = choice(learning_rates)
        model = setup_model_dynamic(num_neurons, input_size, output_size, dropout, activation, activation_output)
        model, history = setup_compile_model(model, optimizer,dados_treino, dados_validacao, learning_rate, loss, epochs, steps_per_epoch, metrics)
        val_loss, val_accuracy = model.evaluate(dados_teste, verbose = 0)
        
        if verbose:
            print("num_neurons\tDropout\tactivation\tactivation_output\tval_loss\tval_accuracy\n")
            print(num_neurons, "\t", dropout, "\t", "\t", activation, "\t", activation_output,"\t", val_loss, "\t", val_accuracy)
        
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_configurations = (num_neurons, dropout, activation, activation_output, optimizer, loss, epochs, steps_per_epoch, learning_rate, metrics)
    print("Best Accuracy: ", best_accuracy) 
    print("Configurations of model: Nª neurons: " + str(best_configurations[0]) + " dropout: "+ str(best_configurations[1]) + " activation: "+ str(best_configurations[2]) + " activation_output: "+ str(best_configurations[3]) + " activation_output: "+ str(best_configurations[4]) + " loss: "+ str(best_configurations[5]) + " epochs: " + str(best_configurations[6]) + " steps_per_epoch: " + str(best_configurations[7])+ " learning rate: " + str(best_configurations[8]) + " metric: " + str(best_configurations[9]))
    return best_configurations

In [ ]:
best_configurations = run_dnn(dados_treino, dados_validacao, dados_teste, 10)

In [ ]:
best_model = setup_model_dynamic(best_configurations[0], 1, 1, best_configurations[1], best_configurations[2], best_configurations[3])
best_model, history = setup_compile_model(best_model, best_configurations[0], dados_treino, dados_validacao, best_configurations[8], best_configurations[5], best_configurations[6], best_configurations[7], best_configurations[9])
scores = best_model.evaluate(dados_teste, verbose = 0)
print("Accuracy: ", scores[1] * 100)

In [ ]:
print(best_model.summary())

In [ ]:
#Visualizar gráficos
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

#Gráfico Accuracy
plt.plot(epochs, accuracy, 'b', color = 'green', label = 'Accuracy Treino')
plt.plot(epochs, val_accuracy, 'b', label = 'Accuracy Validação')
plt.title('Accuracy do treino e da validação')
plt.legend()
plt.figure()

#Gráfico Loss
plt.plot(epochs, loss, 'b', color = 'green', label = 'Loss Treino')
plt.plot(epochs, val_loss, 'b', label = 'Loss Validação')
plt.title('Loss do treino e da validação')
plt.legend()
plt.show()

In [ ]:
def visualize_previsao(img, img_class):
    print("X:",img.shape)
    print("y:",img_class.shape)
    plt.imshow(np.squeeze(img, axis=2))
    plt.show()
    print("label:",img_class)
    print("antes do aumento de mais uma dimensão:",img.shape)
    imagem_tensor = np.expand_dims(img, axis=0)
    print("depois do aumento de mais uma dimensão:",imagem_tensor.shape)
    print("previsão:",best_model.predict(imagem_tensor))
    classes = best_model.predict_classes(imagem_tensor)
    print('Classe prevista:',classes)
    return imagem_tensor
#dados_teste.__getitem__(0)[0], o primeiro 0 é a bacth dos dados, neste caso como dados_teste tem 52 imagens, uma bacth size de 10 vamos ter 6 batchs, podendo este número ir até 5
# O segundo 0 é se queremos os atributos ou as labels (1)
#dados_teste.__getitem__(0)[0][1] vamos buscar na bacth 0 os atributos da imagem 1 

imagem_tensor = visualize_previsao(dados_teste.__getitem__(0)[0][1], dados_teste.__getitem__(0)[1][1])

In [ ]:
camadas_outputs = []
for layer in best_model.layers[:4]:
    print(layer.output.shape)
    camadas_outputs.append(layer.output)

# e depois criamos um modelo que retorna estes outputs dado os inputs do modelo
activation_model = Model(inputs=best_model.input, outputs=camadas_outputs) 
activations = activation_model.predict(imagem_tensor)
first_layer_activation = activations[0]
print(first_layer_activation.shape)
plt.imshow(first_layer_activation[0, :, :, 9], cmap='viridis')
plt.show()

In [ ]:
nome_camadas = []
for camada in best_model.layers[:4]:
    print("Nome da camada: ", camada.name)
    nome_camadas.append(camada.name)

imagens_por_linha = 8 

for nome_camada, ativacao_camada in zip(nome_camadas, activations): # o zip permite iterar simultaneamente em 2 listas
    n_features = ativacao_camada.shape[-1] # Numero de features no feature map, pois é o que está na ultima dimensão
    size = ativacao_camada.shape[1] #O feature map tem shape (1, tamanho, tamanho, numero_features).
    n_linhas = -(-n_features // imagens_por_linha) # Empilha os canais de ativação nesta matriz
    print("nome_camada:",nome_camada)
    print("n_features:",n_features)
    print("size:",size)
    print("n_linhas:",n_linhas)
    display_grid = np.zeros((size * n_linhas, imagens_por_linha * size))
    for col in range(n_linhas): # para fazer o display com 15 imagens por linha
        for lin in range(imagens_por_linha):
            #verificar aqui se a imagem existe
            imagem = ativacao_camada[0,:,:,col * imagens_por_linha + lin] #isto pode dar erro de out-of-range
            imagem -= imagem.mean() # pos-processamento para melhor visualização
            imagem /= imagem.std()
            imagem *= 64
            imagem += 128
            imagem = np.clip(imagem, 0, 255).astype('uint8') #valores <0 ficam 0 e >255 ficam = 255
            display_grid[col * size : (col + 1) * size,lin * size : (lin + 1) * size] = imagem
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(nome_camada)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')        

